# XGB GPU Optuna Optimizer

## For Kaggle AMEX default prediction competition data

https://www.kaggle.com/competitions/amex-default-prediction/data

This competition had some random looking data where no information on columns was given. The notebook is mainly to allow me to copy it for use in Logreg optimization in other tabular data cases.

This assumes XGB (xgboost) and various other Python libraries are installed. And GPU configured. I used a Docker image and Pipfile with all these installed.


In [1]:
#!pip install xgboost

In [1]:
import pandas as pd
import numpy as np

import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tqdm import tqdm
import gc
from sklearn import preprocessing

tqdm.pandas()

import time

N_FOLDS = 5

In [2]:
# https://www.kaggle.com/kyakovlev
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)


def xgboost_amex_metric_mod(predt: np.ndarray, dtrain: xgb.DMatrix):
    y = dtrain.get_label()
    return 'AMEXcustom', 1 - amex_metric_mod(y, predt)

def xgboost_amex_metric_mod2(predt: np.ndarray, dtrain: xgb.DMatrix):
    y = dtrain.get_label()
    return 'AMEXcustom2', amex_metric_mod(y, predt)


In [3]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 


In [4]:
#https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2
from typing import List

def optimize_floats(df: pd.DataFrame) -> pd.DataFrame:
    floats = df.select_dtypes(include=['float64']).columns.tolist()
    df[floats] = df[floats].apply(pd.to_numeric, downcast='float')
    return df

def optimize_ints(df: pd.DataFrame) -> pd.DataFrame:
    ints = df.select_dtypes(include=['int64']).columns.tolist()
    df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
    return df

#this does not seem to work for this dataset, customer id's got duplicated
def optimize_objects(df: pd.DataFrame, datetime_features: List[str]) -> pd.DataFrame:
    for col in df.select_dtypes(include=['object']):
        if col not in datetime_features:
            if not (type(df[col][0])==list):
                num_unique_values = len(df[col].unique())
                num_total_values = len(df[col])
                if float(num_unique_values) / num_total_values < 0.5:
                    df[col] = df[col].astype('category')
        else:
            df[col] = pd.to_datetime(df[col])
    return df

def optimize(df: pd.DataFrame, datetime_features: List[str] = []):
    return optimize_floats(optimize_ints(df))
    #the optimize objects was causing issue with categoricals
    #return optimize_floats(optimize_ints(optimize_objects(df, datetime_features)))

In [5]:
def format_time(seconds):
    seconds = int(seconds)
    minutes = seconds // 60
    hours = minutes // 60
    minutes = minutes % 60
    seconds = seconds % 60
    if hours > 0:
        return f"{hours}h, {minutes}m, {seconds}s"
    if minutes > 0:
        return f"{minutes}m, {seconds}s"
    return f"{seconds}s"
        

In [6]:
def find_new_cat_cols(df_from):
    new_cat_cols = []
    for cat_col in cat_cols:
        new_cat_cols.extend([col for col in df_from.columns if col.startswith(cat_col)])
    return new_cat_cols


# XGB Optuna Optimizer Code

In [7]:
#https://xgboost.readthedocs.io/en/latest/parameter.html

optuna_eval_results = {}
optuna_iteration_index = 0
optuna_losses = []
optuna_accuracies = []
optuna_amexes = []
optuna_parameters = []
optuna_seconds = []

def optuna_objective(trial, X, y, strat, verbosity=-1):
    global optuna_iteration_index
    
    optuna_iteration_index += 1
    #param_grid = {}  # to be filled in later
    param_grid = {
        #'max_depth': trial.suggest_int('max_depth', 2, 10, 1),
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3),
        # nthread defaults to maximum so not setting it
        'subsample': trial.suggest_float('subsample', 0.75, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 0, 5),
        "objective": 'binary:logistic', 
        "max_depth" : -1,
#        'eval_metric': "logloss",
#        'eval_metric': amex_metric_mod,
        'eval_metric': amex_metric_mod,
        'n_estimators': 10000, #500,
        'n_jobs': 16,
        # 'tree_method': 'gpu_hist',
    }
    param = param_grid
    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
        param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 10)#1e-8, 1.0)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
        param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)
    if True:
        # tree_method: 'gpu_hist' causes xgboost to use GPU.
        # https://xgboost.readthedocs.io/en/latest/gpu/
        param_grid["tree_method"] = "gpu_hist"
        param_grid['predictor'] = 'gpu_predictor'
    if None is not None:
        param_grid["scale_pos_weight"] = self.scale_pos_weight

    print(f"STARTING OPTIMIZATION iteration {optuna_iteration_index}, data size: {X.shape}, params={param_grid}")
    #split data into 5 pieces, run the optimizer on only 2 of them for speed
    N_FOLDS_RUN = N_FOLDS
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=69)
#    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1121218)

    cv_scores = np.empty(N_FOLDS_RUN)
    cv_accuracies = np.empty(N_FOLDS_RUN)
    cv_amexes = np.empty(N_FOLDS_RUN)
    
    time_start_all_folds = time.time()
    acc_score_total = 0
    n_classes = 1
    train_preds = np.zeros((X.shape[0], n_classes))
    amex_total = 0
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, strat)):
        if idx >= N_FOLDS_RUN:
            break
        time_start_this_fold = time.time()
        print(f"=== STARTING FOLD {idx+1}/{N_FOLDS_RUN} ({N_FOLDS}) ===")
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
#        evalset = [(X_train, y_train), (X_test,y_test)]
        evalset = [(X_test,y_test)]
        
        print("creating classifier")
        model = xgb.XGBClassifier(**param_grid, enable_categorical = True)
        print("fitting")
        
        es = xgb.callback.EarlyStopping(
            rounds=100,
            min_delta=1e-3,
    #        save_best=True,
            maximize=True,
    #        data_name="valid",
            metric_name="amex_metric_mod",
        )
        cbs = [es]
        
        model.fit(
            X_train,
            y_train,
#            eval_metric: xgboost_amex_metric_mod,
            verbose=100,
            eval_set=evalset,
            #early_stopping_rounds=100,
            callbacks=cbs
        )

        print("predicting")
        preds = model.predict_proba(X_test, iteration_range=(0, model.best_iteration + 1))

        preds_true_amex = preds[:, 1]
        preds_true = preds[:, 1].flatten()
        preds_true.shape = (y_test.shape[0], 1)
        amex_y_test = np.array(y_test)
        amex_y_test.shape = (y_test.shape[0], 1)
        np.add.at(train_preds, test_idx, preds_true)
        print(f"count zerO: {np.count_nonzero(train_preds)}")

        preds_this_round = (preds >= 0.5)[:,1]

        acc_score = accuracy_score(y_test, preds_this_round)
        acc_score_total += acc_score

        amex_score = amex_metric_mod(y_test, preds_true_amex)
        amex_total += amex_score

        log_loss_fold = log_loss(y_test, preds)
        cv_scores[idx] = log_loss_fold
        cv_accuracies[idx] = acc_score
        cv_amexes[idx] = amex_score
        print(f"amex: {amex_score}")

        print(f"=== FINISHED FOLD {idx+1}/{N_FOLDS}, log loss={log_loss_fold}, accuracy={acc_score}, amex={amex_score} ===")
        time_total_this_fold = time.time() - time_start_this_fold
        print(f"time to run this fold: {format_time(time_total_this_fold)}")        

        if amex_score < 0.78:
            print("low score on amex, dropping loop")
            return amex_score
   
    time_total_all_folds = time.time() - time_start_all_folds        
    print(f"time to run folds: {format_time(time_total_all_folds)}")
    
    iteration_log_loss = np.mean(cv_scores)
    iteration_accuracy = np.mean(cv_accuracies)
    iteration_amex = np.mean(cv_amexes)
    optuna_losses.append(iteration_log_loss)
    optuna_amexes.append(iteration_amex)
    optuna_accuracies.append(iteration_accuracy)
    optuna_parameters.append(param_grid)
    optuna_seconds.append(int(time_total_all_folds))
    print(f"iteration logloss: {iteration_log_loss}, iteration acc: {iteration_accuracy}, Iteration amex: {iteration_amex}")
    return iteration_amex #iteration_log_loss


# Load Data

In [8]:
df_train = pd.read_parquet("train_lagged.parquet", engine="pyarrow")
df_train.head()

,customer_ID,S_2_first,S_2_mean,S_2_std,S_2_min,S_2_max,S_2_last,P_2_first,P_2_mean,P_2_std,...,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique,target,fake_splitter
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.0,0.0,0,0,0,0.94,0.933846,0.024337,...,13,0,0,1,13,7,7,1,0,True
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,0.0,0.0,0,0,0,0.93,0.899231,0.022899,...,13,0,0,1,13,7,7,1,0,True
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.0,0.0,0,0,0,0.88,0.877692,0.027735,...,13,0,0,1,13,7,7,1,0,True
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,0.0,0.0,0,0,0,0.57,0.597692,0.018777,...,13,0,0,1,13,3,4,3,0,True
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.0,0.0,0,0,0,0.94,0.893077,0.042502,...,13,2,2,1,13,7,7,1,0,True


In [9]:
df_test = pd.read_parquet("test_lagged.parquet", engine="pyarrow")
df_test.head()

,customer_ID,S_2_first,S_2_mean,S_2_std,S_2_min,S_2_max,S_2_last,P_2_first,P_2_mean,P_2_std,...,D_64_last,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0,0.0,0.0,0,0,0,0.63,0.602222,0.019861,...,4,2,9,0,0,1,9,0,7,2
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0,0.0,0.0,0,0,0,0.89,0.860769,0.030947,...,1,1,13,0,0,1,13,7,7,1
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0,0.0,0.0,0,0,0,0.77,0.748462,0.061893,...,4,2,13,2,2,1,13,7,5,2
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0,0.0,0.0,0,0,0,0.51,0.474615,0.026962,...,3,1,13,0,0,1,13,6,6,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0,0.0,0.0,0,0,0,0.29,0.323846,0.050751,...,3,2,13,0,0,1,13,7,6,2


In [10]:
#special combo-column created for stratification
strat = df_train["fake_splitter"]


In [11]:
cols_to_drop = [col for col in df_train.columns if "fake" in col]
cols_to_drop

['fake_splitter']

In [12]:
df_train = df_train.drop(cols_to_drop, axis=1)

In [13]:
X = df_train.drop(["customer_ID", "target"], axis=1)
y = df_train["target"]

In [14]:
stratzip = zip(strat, y)
#stratified k-fold only supports single binary value, tuple stratification breaks it
stratzip = [f"{a}-{b}" for (a,b) in stratzip]
#stratzip = list(stratzip)

In [25]:
cat_cols = find_new_cat_cols(df_train)

In [26]:
num_cols = [col for col in df_train.columns if col not in cat_cols if col != "customer_ID"]

# Run Optuna Optimizer

In [15]:
optuna_eval_results = {}
optuna_iteration_index = 0
optuna_losses = []
optuna_accuracies = []
optuna_amexes = []
optuna_parameters = []
optuna_seconds = []

#study = optuna.create_study(direction="minimize", study_name="XGB Classifier")
study = optuna.create_study(direction="maximize", study_name="XGB Classifier")
func = lambda trial: optuna_objective(trial, X, y, stratzip)
study.optimize(func, n_trials=20) #more folds generally, just protoing with 20

[I 2022-08-28 13:30:17,165] A new study created in memory with name: XGB Classifier


STARTING OPTIMIZATION iteration 1, data size: (458913, 1580), params={'booster': 'gblinear', 'lambda': 0.001476479059081602, 'alpha': 0.0002072147510625708, 'learning_rate': 0.1500858515816118, 'subsample': 0.824727326751916, 'colsample_bytree': 0.7901230710210116, 'colsample_bylevel': 0.8725667048715013, 'min_child_weight': 5, 'objective': 'binary:logistic', 'max_depth': -1, 'eval_metric': <function amex_metric_mod at 0x7fa3788d18b0>, 'n_estimators': 10000, 'n_jobs': 16, 'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor'}
=== STARTING FOLD 1/5 (5) ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[13:30:20] WARNING: ../src/learner.cc:627: 
Parameters: { "colsample_bylevel", "colsample_bytree", "max_depth", "min_child_weight", "predictor", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-logloss:0.31746	validation_0-amex_metric_mod:0.73048
[100]	validation_0-logloss:0.23064	validation_0-amex_metric_mod:0.77948
[159]	validation_0-logloss:0.23046	validation_0-amex_metric_mod:0.77979
predicting


[I 2022-08-28 13:31:13,977] Trial 0 finished with value: 0.7797884795278291 and parameters: {'booster': 'gblinear', 'lambda': 0.001476479059081602, 'alpha': 0.0002072147510625708, 'learning_rate': 0.1500858515816118, 'subsample': 0.824727326751916, 'colsample_bytree': 0.7901230710210116, 'colsample_bylevel': 0.8725667048715013, 'min_child_weight': 5}. Best is trial 0 with value: 0.7797884795278291.


...

[0]	validation_0-logloss:0.65444	validation_0-amex_metric_mod:0.71387
[100]	validation_0-logloss:0.22525	validation_0-amex_metric_mod:0.78560
[200]	validation_0-logloss:0.21922	validation_0-amex_metric_mod:0.79103
[300]	validation_0-logloss:0.21777	validation_0-amex_metric_mod:0.79318
[394]	validation_0-logloss:0.21730	validation_0-amex_metric_mod:0.79329
predicting
count zerO: 367131
amex: 0.7935509015344215
=== FINISHED FOLD 4/5, log loss=0.21783810210897497, accuracy=0.9032163169248872, amex=0.7935509015344215 ===
time to run this fold: 37s
=== STARTING FOLD 5/5 (5) ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.65460	validation_0-amex_metric_mod:0.70962
[100]	validation_0-logloss:0.22755	validation_0-amex_metric_mod:0.77748
[200]	validation_0-logloss:0.22186	validation_0-amex_metric_mod:0.78399
[300]	validation_0-logloss:0.22049	validation_0-amex_metric_mod:0.78582
[392]	validation_0-logloss:0.22003	validation_0-amex_metric_mod:0.78610
predicting


[I 2022-08-28 17:30:41,147] Trial 19 finished with value: 0.7899273142794456 and parameters: {'booster': 'gbtree', 'lambda': 0.001147016685108034, 'alpha': 0.12271449814510488, 'learning_rate': 0.059852978433669615, 'subsample': 0.9430057322345442, 'colsample_bytree': 0.6027379962480811, 'colsample_bylevel': 0.3842654047202902, 'min_child_weight': 1, 'max_depth': 7, 'eta': 4.9314281070766694e-05, 'gamma': 1.4178172184173403e-08, 'grow_policy': 'depthwise'}. Best is trial 19 with value: 0.7899273142794456.


count zerO: 458913
amex: 0.7859337278485907
=== FINISHED FOLD 5/5, log loss=0.22050629699923596, accuracy=0.9024427447647687, amex=0.7859337278485907 ===
time to run this fold: 38s
time to run folds: 3m, 0s
iteration logloss: 0.21915403647589266, iteration acc: 0.9031123533967328, Iteration amex: 0.7899273142794456


In [16]:
sum(optuna_seconds)

7654

# Add Metadata to Optimizer Results for Analysis

In [17]:
df_params = pd.DataFrame(optuna_parameters)
df_params["loss"] = optuna_losses
df_params["accuracy"] = optuna_accuracies
df_params["amex"] = optuna_amexes
df_params["time"] = optuna_seconds


In [18]:
df_params.to_csv("optimizer_results_xgb_lagged_seed2.csv")

In [19]:
df_params = pd.read_csv("optimizer_results_xgb_lagged_seed2.csv")

# Train and Predict After Optimization (Code)

In [20]:
def train_and_predict(optuna_params, df_from, strat):
    X = df_from.drop(["customer_ID", "target"], axis=1)
    y = df_from["target"]

    verbosity=50 #print status every 50 iteration of tree building
    optuna_eval_results = {}

    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=69)
#    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1121218)

    cv_scores = np.empty(N_FOLDS)
    cv_accuracies = np.empty(N_FOLDS)
    time_start_all_folds = time.time()
    acc_score_total = 0
    amex_score_total = 0
    models = []

    n_classes = 1
    train_preds = np.zeros((X.shape[0], n_classes))
    #balanced_preds = np.empty([X.shape[0], n_classes], dtype = float)

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, strat)):

        
        time_start_this_fold = time.time()
        print(f"=== STARTING FOLD {idx+1}/{N_FOLDS} ===")
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
#        evalset = [(X_train, y_train), (X_test,y_test)]
        evalset = [(X_test,y_test)]
        
        print("creating classifier")
        model = xgb.XGBClassifier(**optuna_params, enable_categorical = True)

        es = xgb.callback.EarlyStopping(
            rounds=100,
            min_delta=1e-3,
            maximize=True,
            metric_name="amex_metric_mod",
        )
        cbs = [es]
        
        print("fitting")
        model.fit(
            X_train,
            y_train,
            verbose=20,
            eval_set=evalset,
            callbacks=cbs,
        )
        
        print(f"predicting, best round: {model.best_iteration+1}")
        preds = model.predict_proba(X_test, iteration_range=(0, model.best_iteration + 1))

        preds_true_amex = preds[:, 1]
        preds_true = preds[:, 1].flatten()
        preds_true.shape = (y_test.shape[0], 1)
        np.add.at(train_preds, test_idx, preds_true)
        print(f"count zerO: {np.count_nonzero(train_preds)}")
        
        amex_score = amex_metric_mod(y_test, preds_true_amex)
        amex_score_total += amex_score
        
        preds_this_round = (preds >= 0.5)[:,1]
        
        acc_score = accuracy_score(y_test, preds_this_round)
        acc_score_total += acc_score

        log_loss_fold = log_loss(y_test, preds)
        cv_scores[idx] = log_loss_fold
        cv_accuracies[idx] = acc_score
        print(f"=== FINISHED FOLD {idx+1}/{N_FOLDS}, log loss={log_loss_fold}, accuracy={acc_score}, amex={amex_score} ===")
        time_total_this_fold = time.time() - time_start_this_fold
        print(f"time to run this fold: {format_time(time_total_this_fold)}")
        models.append(model)
        
    print(f"amex avg: {amex_score_total/N_FOLDS}")
        
    return models, train_preds


# Pick Best Optimizer Result, Compare Optimizer Runs

In [29]:
best_idx = np.argmax(df_params["amex"])
best_idx

7

In [30]:
optuna_params = df_params.iloc[best_idx]
optuna_params = dict(optuna_params)
optuna_params


{'Unnamed: 0': 7,
 'booster': 'gbtree',
 'lambda': 0.001147016685108,
 'alpha': 0.1227144981451048,
 'learning_rate': 0.0598529784336696,
 'subsample': 0.9430057322345442,
 'colsample_bytree': 0.6027379962480811,
 'colsample_bylevel': 0.3842654047202902,
 'min_child_weight': 1,
 'objective': 'binary:logistic',
 'max_depth': 7,
 'eval_metric': '<function amex_metric_mod at 0x7fa3788d18b0>',
 'n_estimators': 10000,
 'n_jobs': 16,
 'eta': 4.931428107076669e-05,
 'gamma': 1.4178172184173404e-08,
 'grow_policy': 'depthwise',
 'sample_type': nan,
 'normalize_type': nan,
 'rate_drop': nan,
 'skip_drop': nan,
 'tree_method': 'gpu_hist',
 'predictor': 'gpu_predictor',
 'loss': 0.2191540364758926,
 'accuracy': 0.9031123533967328,
 'amex': 0.7899273142794456,
 'time': 180}

In [31]:
del optuna_params['Unnamed: 0']

In [32]:
#optuna_params

In [33]:
#method references dont seem to come well off the optimizer, so have to re-set it here
optuna_params["eval_metric"] = amex_metric_mod


# Train Classifier with Best Params, Predict Train Set via Folds

In [34]:
final_models, final_preds = train_and_predict(optuna_params, df_train, strat)

=== STARTING FOLD 1/5 ===
creating classifier
fitting


/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[17:30:47] WARNING: ../src/learner.cc:627: 
Parameters: { "accuracy", "amex", "loss", "normalize_type", "rate_drop", "sample_type", "skip_drop", "time" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-logloss:0.65456	validation_0-amex_metric_mod:0.70922
[20]	validation_0-logloss:0.32878	validation_0-amex_metric_mod:0.75854
[40]	validation_0-logloss:0.25687	validation_0-amex_metric_mod:0.76868
[60]	validation_0-logloss:0.23616	validation_0-amex_metric_mod:0.77444
[80]	validation_0-logloss:0.22887	validation_0-amex_metric_mod:0.77888
[100]	validation_0-logloss:0.22566	validation_0-amex_metric_mod:0.78232
[120]	validation_0-logloss:0.22386	validation_0-amex_metric_mod:0.78475
[140]	validation_0-logloss:0.22251	validation_0-amex_metric_

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[17:31:22] WARNING: ../src/learner.cc:627: 
Parameters: { "accuracy", "amex", "loss", "normalize_type", "rate_drop", "sample_type", "skip_drop", "time" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-logloss:0.65451	validation_0-amex_metric_mod:0.71436
[20]	validation_0-logloss:0.32884	validation_0-amex_metric_mod:0.76164
[40]	validation_0-logloss:0.25694	validation_0-amex_metric_mod:0.77089
[60]	validation_0-logloss:0.23592	validation_0-amex_metric_mod:0.77564
[80]	validation_0-logloss:0.22859	validation_0-amex_metric_mod:0.77912
[100]	validation_0-logloss:0.22498	validation_0-amex_metric_mod:0.78292
[120]	validation_0-logloss:0.22284	validation_0-amex_metric_mod:0.78527
[140]	validation_0-logloss:0.22155	validation_0-amex_metric_

...

[17:31:56] WARNING: ../src/learner.cc:627: 
Parameters: { "accuracy", "amex", "loss", "normalize_type", "rate_drop", "sample_type", "skip_drop", "time" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-logloss:0.65457	validation_0-amex_metric_mod:0.70950
[20]	validation_0-logloss:0.32971	validation_0-amex_metric_mod:0.75525
[40]	validation_0-logloss:0.25790	validation_0-amex_metric_mod:0.76529
[60]	validation_0-logloss:0.23686	validation_0-amex_metric_mod:0.77114
[80]	validation_0-logloss:0.22935	validation_0-amex_metric_mod:0.77702
[100]	validation_0-logloss:0.22585	validation_0-amex_metric_mod:0.77990
[120]	validation_0-logloss:0.22383	validation_0-amex_metric_mod:0.78255
[140]	validation_0-logloss:0.22247	validation_0-amex_metric_

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[17:32:31] WARNING: ../src/learner.cc:627: 
Parameters: { "accuracy", "amex", "loss", "normalize_type", "rate_drop", "sample_type", "skip_drop", "time" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-logloss:0.65444	validation_0-amex_metric_mod:0.71387
[20]	validation_0-logloss:0.32918	validation_0-amex_metric_mod:0.76407
[40]	validation_0-logloss:0.25749	validation_0-amex_metric_mod:0.77218
[60]	validation_0-logloss:0.23636	validation_0-amex_metric_mod:0.77695
[80]	validation_0-logloss:0.22884	validation_0-amex_metric_mod:0.78154
[100]	validation_0-logloss:0.22525	validation_0-amex_metric_mod:0.78560
[120]	validation_0-logloss:0.22313	validation_0-amex_metric_mod:0.78687
[140]	validation_0-logloss:0.22177	validation_0-amex_metric_

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[17:33:08] WARNING: ../src/learner.cc:627: 
Parameters: { "accuracy", "amex", "loss", "normalize_type", "rate_drop", "sample_type", "skip_drop", "time" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-logloss:0.65460	validation_0-amex_metric_mod:0.70962
[20]	validation_0-logloss:0.32995	validation_0-amex_metric_mod:0.75421
[40]	validation_0-logloss:0.25866	validation_0-amex_metric_mod:0.76419
[60]	validation_0-logloss:0.23818	validation_0-amex_metric_mod:0.76895
[80]	validation_0-logloss:0.23092	validation_0-amex_metric_mod:0.77355
[100]	validation_0-logloss:0.22755	validation_0-amex_metric_mod:0.77748
[120]	validation_0-logloss:0.22540	validation_0-amex_metric_mod:0.77990
[140]	validation_0-logloss:0.22396	validation_0-amex_metric_

In [35]:
df_preds = pd.DataFrame()
df_preds["customer_ID"] = df_train["customer_ID"]
df_preds["prediction"] = final_preds

In [36]:
#calculate achieved competition metric
amex_metric_mod(y, final_preds[:, 0])     

0.7897820616111997

In [37]:
df_preds.to_csv("predictions_xgb_lagged.csv")

# Prediction Kaggle Test Set with Best Params, Over N Folds

Could increase folds here from the Optimizer search if useful

In [38]:
X = df_test.reset_index(drop=True).drop(["customer_ID"], axis=1)


In [39]:
from tqdm import tqdm

n_classes = 1
sub_preds = np.zeros((X.shape[0], n_classes))

#tqdm.auto()
tqdm.pandas()

for model in tqdm(final_models): #lqtm
    preds = model.predict_proba(X, iteration_range=(0, model.best_iteration + 1))
    preds_true = preds[:, 1]
    preds_true.shape = (X.shape[0], 1)
    sub_preds += preds_true
sub_preds /= N_FOLDS


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:38<00:00,  7.79s/it]


In [40]:
submission = pd.DataFrame()
submission["customer_ID"] = df_test.reset_index()["customer_ID"]
submission["prediction"] = sub_preds
submission

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.033213
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.001080
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.028299
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.244375
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.852290
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.021676
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.862490
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.555502
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.354948


In [41]:
submission.to_csv("submission_xgb_lagged.csv", index=False)

In [42]:
#df_all["D_50"].quantile(0.99)

In [43]:
pd.reset_option("display.max_rows")
pd.reset_option("display.min_rows")
pd.reset_option("display.max_columns")
